In [ ]:
# Import key librarys
import pandas as pd
import numpy as np

In [ ]:
# Define function to tidy data
def tidy(df):
    
    #### Fix usual issues with all strings
    
    # Capitalise headers
    #df.columns = df.columns.astype(str).str.upper()
    df.columns = df.columns.set_levels([level.str.upper() for level in df.columns.levels])
    
    # Capitalise columns
    df = df.map(lambda x: x.upper() if type(x) is str else x)

    # Strip whitespace
    df = df.map(lambda x: x.strip() if type(x) is str else x)

    # Remove parenthesis
    df = df.map(lambda x: x.replace('(', '') if type(x) is str else x)
    df = df.map(lambda x: x.replace(')', '') if type(x) is str else x)
    
    # Remove dashes
    df = df.map(lambda x: x.replace('-', '') if type(x) is str else x)
    
    # Remove full stops
    df = df.map(lambda x: x.replace('.', '') if type(x) is str else x)
    
    # Remove commas
    df = df.map(lambda x: x.replace(',', '') if type(x) is str else x)
    
    # Remove linebreaks
    df = df.map(lambda x: x.replace('\n', '') if type(x) is str else x)

    # Replace annoying substrings
    df = df.map(lambda x: x.replace(' AND ', ' & ') if type(x) is str else x)
    df = df.map(lambda x: x.replace(' – ', ' - ') if type(x) is str else x)
    df = df.map(lambda x: x.replace(' / ', '/') if type(x) is str else x)
    df = df.map(lambda x: x.replace('/ ', '/') if type(x) is str else x)
    df = df.map(lambda x: x.replace(' /', '/') if type(x) is str else x)
    
    # Strip whitespace again
    df = df.map(lambda x: x.strip() if type(x) is str else x)
    
    # Remove double spaces
    df = df.map(lambda x: x.replace('  ', ' ') if type(x) is str else x)
    
    # Drop rows with minimum number of 2 non-null values
    df = df.dropna(thresh=2)
    
    return df

# Read in data from 2016/17 to 2022/23

In [ ]:
# Create an empty list to store dataframes
frames = []

In [ ]:
# Create a list of years
years = ['16 17', '17 18', '19 20', '20 21', '21 22', '22 23']

# Loop through years
for year in years:

    # Read in xlsx
    df = pd.read_excel('./foi/ECS ' + year + '.xlsx', header = [0,1])
    
    # Tidy up df
    df = tidy(df)
    
    ### More tidying
    
    # Drop Water columns
    df = df.drop('WATER', axis = 1, level = 0)

    # Drop Site Group Name column (ignore errors as column not in 22/23 file)
    df = df.drop('SITE GROUP NAME', axis = 1, level = 1, errors='ignore')

    # Rename columns
    df = df.rename(columns =
                   {'PRESENT CONSUMPTION': 'KWH', 
                    'SUM PRESENT NET COST': '£',
                    'ACCOUNT COMMODITY': 'LOCATION',
                    'SITE NAME': ''})

    # Convert Oil litres into Oil kWh (assume kerosine with ratio of 10.35)
    df.loc[:, ('OIL','KWH')] *= 10.35
    
    ### Create new columns with £/kWh

    # Replace zeros with NaN to allow division
    #df.replace(0, np.nan, inplace = True)
    
    # Create list of level one headers
    headers = ['BIOMASS', 'ELECTRICITY', 'GAS', 'OIL', 'PROPANE']
    
    # Loop through headers
    
    for header in headers:
        
        # Create new column with values of Cost / kWh
        
        df.loc[:, (header,'£/KWH')] = df[header]['£'] / df[header]['KWH']

    ### Convert into long format and append to list

    # Turn into long format
    df = pd.melt(df, id_vars = 'LOCATION', var_name = ['TYPE', 'UNIT'], value_name = 'AMOUNT')

    # Swap space for /
    year = year.replace(' ', '/')

    # Add year
    df['YEAR'] = '20' + year
    
    # Append to df list
    frames.append(df)

# Read in older data

In [ ]:
# Read in oil data
df = pd.read_excel('./foi/ECS Oil Annualised Data to Mar 2016.xls', header = [0,1])

# Tidy up df
df = tidy(df)

# Drop empty columns
#df = df.dropna(how = "all", axis=1)

# Drop columns with substring 'Units'
df = df[df.columns.drop(list(df.filter(regex='UNITS')))]

# Drop duplicate columns
df = df.T.drop_duplicates().T

# Add '20' to beginning of column years (second level of multi-index)
df = df.rename(columns=lambda x: '20' + x, level=1)

# Change blackslash to forward slash (second level of multi-index)
df.columns = df.columns.set_levels(df.columns.levels[1].str.replace('\\\\', '/', regex=True), level=1)

# Rename columns
df = df.rename(columns =
               {'CONSUMPTION (KWH)': 'KWH', 
                'COST (KWH)': '£',
                'UNNAMED: 0_LEVEL_0': 'LOCATION',
                '20SITE': ''})

# Convert Oil litres into Oil kWh (assume kerosine with ratio of 10.35)
df['KWH'] *= 10.35

### Create new columns with £/kWh

# Replace zeros with NaN to allow division
df.replace(0, np.nan, inplace = True)

# Create new df with values of Cost / kWh columns
dfn = df['£'] / df['KWH']

# Add column level to index = £/kWh so that it matches format of orginal df
dfn.columns = pd.MultiIndex.from_product([['£/KWH'], dfn.columns])

# Join dfn to orginal df
df = df.join(dfn)

### Convert into long format and append to list

# Turn into long format
df = pd.melt(df, id_vars = 'LOCATION', var_name = ['UNIT', 'YEAR'], value_name = 'AMOUNT')

# Add type
df['TYPE'] = 'OIL'

# Append to df list
frames.append(df)

In [ ]:
# Read in electricity data
df = pd.read_excel('./foi/ECS Elec Annualised Data to Mar 2016.xls', header = [0,1])

# Tidy up df
df = tidy(df)

# Drop empty columns
#df = df.dropna(how = "all", axis=1)

# Drop columns with substring 'Units'
df = df[df.columns.drop(list(df.filter(regex='UNITS')))]

# Drop duplicate columns
df = df.T.drop_duplicates().T

# Add '20' to beginning of column years (second level of multi-index)
df = df.rename(columns=lambda x: '20' + x, level=1)

# Change blackslash to forward slash (second level of multi-index)
df.columns = df.columns.set_levels(df.columns.levels[1].str.replace('\\\\', '/', regex=True), level=1)

# Rename columns
df = df.rename(columns =
               {'CONSUMPTION (KWH)': 'KWH', 
                'COST (KWH)': '£',
                'UNNAMED: 0_LEVEL_0': 'LOCATION',
                '20SITE': ''})

### Create new columns with £/kWh

# Replace zeros with NaN to allow division
df.replace(0, np.nan, inplace = True)

# Create new df with values of Cost / kWh columns
dfn = df['£'] / df['KWH']

# Add column level to index = £/kWh so that it matches format of orginal df
dfn.columns = pd.MultiIndex.from_product([['£/KWH'], dfn.columns])

# Join dfn to orginal df
df = df.join(dfn)

### Convert into long format and append to list

# Turn into long format
df = pd.melt(df, id_vars = 'LOCATION', var_name = ['UNIT', 'YEAR'], value_name = 'AMOUNT')

# Add type
df['TYPE'] = 'ELECTRICITY'

# Append to df list
frames.append(df)

In [ ]:
# Read in biomass data
df = pd.read_excel('./foi/ECS Biomass Annualised Data to Mar 2016.xls', header = [0,1])

# Tidy up df
df = tidy(df)

# Drop empty columns
#df = df.dropna(how = "all", axis=1)

# Drop columns with substring 'Units'
df = df[df.columns.drop(list(df.filter(regex='UNITS')))]

# Drop duplicate columns
df = df.T.drop_duplicates().T

# Add '20' to beginning of column years (second level of multi-index)
df = df.rename(columns=lambda x: '20' + x, level=1)

# Change blackslash to forward slash (second level of multi-index)
df.columns = df.columns.set_levels(df.columns.levels[1].str.replace('\\\\', '/', regex=True), level=1)

# Rename columns
df = df.rename(columns =
               {'CONSUMPTION (KWH)': 'KWH', 
                'COST (KWH)': '£',
                'UNNAMED: 0_LEVEL_0': 'LOCATION',
                '20SITE': ''})

### Create new columns with £/kWh

# Replace zeros with NaN to allow division
df.replace(0, np.nan, inplace = True)

# Create new df with values of Cost / kWh columns
dfn = df['£'] / df['KWH']

# Add column level to index = £/kWh so that it matches format of orginal df
dfn.columns = pd.MultiIndex.from_product([['£/KWH'], dfn.columns])

# Join dfn to orginal df
df = df.join(dfn)

### Convert into long format and append to list

# Turn into long format
df = pd.melt(df, id_vars = 'LOCATION', var_name = ['UNIT', 'YEAR'], value_name = 'AMOUNT')

# Add type
df['TYPE'] = 'BIOMASS'

# Append to df list
frames.append(df)

In [ ]:
# Read in gas data
df = pd.read_excel('./foi/ECS Gas Annualised Data to Mar 2016.xls', header = [0,1])

# Tidy up df
df = tidy(df)

# Drop empty columns
#df = df.dropna(how = "all", axis=1)

# Drop columns with substring 'Units'
df = df[df.columns.drop(list(df.filter(regex='UNITS')))]

# Drop duplicate columns
df = df.T.drop_duplicates().T

# Add '20' to beginning of column years (second level of multi-index)
df = df.rename(columns=lambda x: '20' + x, level=1)

# Change blackslash to forward slash (second level of multi-index)
df.columns = df.columns.set_levels(df.columns.levels[1].str.replace('\\\\', '/', regex=True), level=1)

# Rename columns
df = df.rename(columns =
               {'CONSUMPTION (KWH)': 'KWH', 
                'COST (KWH)': '£',
                'UNNAMED: 0_LEVEL_0': 'LOCATION',
                '20SITE': ''})

### Create new columns with £/kWh

# Replace zeros with NaN to allow division
df.replace(0, np.nan, inplace = True)

# Create new df with values of Cost / kWh columns
dfn = df['£'] / df['KWH']

# Add column level to index = £/kWh so that it matches format of orginal df
dfn.columns = pd.MultiIndex.from_product([['£/KWH'], dfn.columns])

# Join dfn to orginal df
df = df.join(dfn)

### Convert into long format and append to list

# Turn into long format
df = pd.melt(df, id_vars = 'LOCATION', var_name = ['UNIT', 'YEAR'], value_name = 'AMOUNT')

# Add type
df['TYPE'] = 'GAS'

# Append to df list
frames.append(df)

In [ ]:
df.head()

# Export data

In [ ]:
# Concat frames together
dft = pd.concat(frames)

### Remove dodgy rows

# Convert any infinite values into NaN
dft.replace([np.inf, -np.inf], np.nan, inplace=True)

# Remove any rows with null values
dft = dft.dropna(how = 'any', axis = 0)

# Remove rows with zeros
dft = dft[dft.AMOUNT != 0]

# Drop rows with Location = Total
dft = dft[dft.LOCATION != 'TOTAL']

### Tidy up names

# Replace 'PS' with 'Primary School' and 'HS' with 'High School' in location column
dft['LOCATION']= dft["LOCATION"].replace('PS', 'PRIMARY SCHOOL', regex = True)
dft['LOCATION']= dft["LOCATION"].replace('HS', 'HIGH SCHOOL', regex = True)

# Remove severeal suffixes and prefixes
dft.LOCATION = dft.LOCATION.str.removesuffix('OLD')
dft.LOCATION = dft.LOCATION.str.removesuffix('NEW')
dft.LOCATION = dft.LOCATION.str.removesuffix('MOTHBALLED')
dft.LOCATION = dft.LOCATION.str.removesuffix('FORMS PART')
dft.LOCATION = dft.LOCATION.str.removesuffix('NOT PPP')
dft.LOCATION = dft.LOCATION.str.removeprefix('ZHC')
dft.LOCATION = dft.LOCATION.str.removeprefix('XX')

# Strip whitespace again
dft = dft.map(lambda x: x.strip() if type(x) is str else x)

# Rename columns
dft['LOCATION'] = dft['LOCATION'].replace(
           {'CAOL CAMPUS': 'CAOL JOINT CAMPUS',
            'WICK COMMUNITY CAMPUS': 'WICK JOINT CAMPUS',
            'ELGIN RESIDENCE FORMERLY ELGIN HOSTEL': 'ELGIN RESIDENCE',
            'ELGIN HOSTEL': 'ELGIN RESIDENCE',
            'FT AUGUSTUS': 'FORT AUGUSTUS',
            'MERKINCH NURSERY SCHOOL': 'MERKINCH FAMILY CENTRE',
            'BRORA INTERPRETATIVE CENTRE': 'BRORA HERITAGE CENTRE',
            'MORVEN YOUTH CLUB': 'CASTLETOWN MORVEN YOUTH CLUB',
            'ROSEMARKIE PAVILION': 'ROSEMARKIE BEACH AMENITIES',
            'FERRY BRAE PAVILION NORTH KESSOCK': 'FERRY BRAE SPORTS PAVILION',
            'SPEAN BRIDGE PRIMARY DEMOUNTABLE UNIT': 'SPEAN BRIDGE PRIMARY SCHOOL DEMOUNTABLE UNIT',
            'DORNOCH ACADEMY & PRIMARY': 'DORNOCH ACADEMY & PRIMARY SCHOOL'
           })

In [ ]:
# Sort values and reset index
dft = dft.sort_values(['YEAR', 'LOCATION', 'TYPE'], ascending = [True, True, True]).reset_index(drop=True)

# Export to .csv
dft.to_csv('./csvs/highlandEnergyNew.csv', index = False)

In [ ]:
dft.info()

In [ ]:
dft.head()

In [ ]:
dft.loc[dft['LOCATION'] == 'ACHFARY SCHOOL HOUSE']